In [70]:
import evaluate
import pickle
import time

from numpy import argmax
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer

# Load DatasetDict-s for Few-Shot Learning

In [42]:
dd_dst_paths = [
                         '../dataset/scnd_thrd_train_frst_test_dd.pkl', 
                         '../dataset/frst_thrd_train_scnd_test_dd.pkl', 
                         '../dataset/frst_scnd_train_thrd_test_dd.pkl',
                         '../dataset/all_papers_train_dd.pkl'
                    ]

## Train: 2,3
## Test: 1

In [43]:
##########################################

## Train: 1,3
## Test: 2

In [44]:
##########################################

## Train: 1,2
## Test: 3

In [45]:
with open (dd_dst_paths[2], 'rb') as file:
    frst_scnd_train_thrd_test_dd = pickle.load(file)

## Check the loaded DatasetDict

In [46]:
frst_scnd_train_thrd_test_dd

DatasetDict({
    train: Dataset({
        features: ['paper', 'tokens', 'labels', 'tags'],
        num_rows: 1117
    })
    test: Dataset({
        features: ['paper', 'tokens', 'labels', 'tags'],
        num_rows: 320
    })
})

In [47]:
frst_scnd_train_thrd_test_dd['test'][9]['tokens']

['The',
 'activation',
 'functions',
 'of',
 'the',
 'LSTM',
 'layers',
 'were',
 'tanh',
 '12',
 'in',
 'the',
 'proposed',
 'architecture',
 '.']

In [48]:
for feature in frst_scnd_train_thrd_test_dd['train'].features:
    print(frst_scnd_train_thrd_test_dd['train'][199][feature], '\n')

0 

['5.4', 'Ensemble', 'of', 'trees', 'using', 'LPBoost', 'Boosting', 'is', 'another', 'ensemble', 'method', 'used', 'to', 'enhance', 'the', 'performance', 'of', 'weak', 'learners', 'i.e.', 'trees', '.'] 

[0, 1, 2, 2, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

['O', 'B-ML', 'I-ML', 'I-ML', 'O', 'B-ML', 'I-ML', 'O', 'O', 'B-ML', 'I-ML', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] 



In [49]:
for feature in frst_scnd_train_thrd_test_dd['test'].features:
    print(frst_scnd_train_thrd_test_dd['test'][100][feature], '\n')

2 

['The', 'input', 'goes', 'through', '122', 'two', 'LSTM', 'layers', 'and', 'then', 'a', 'ﬂatten', 'layer', 'is', 'attached', 'to', 'it', 'where', 'the', 'second', 'LSTM', '123', 'layer', 'was', 'bidirectional', 'layer', '.'] 

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] 

['O', 'O', 'O', 'O', 'O', 'O', 'B-ML', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ML', 'O', 'O', 'O', 'O', 'O', 'O'] 



# Match up words with their NER tags

In [50]:
label_names = ['O', 'B-ML', 'I-ML']

In [51]:
words = frst_scnd_train_thrd_test_dd["train"][199]["tokens"]
labels = frst_scnd_train_thrd_test_dd["train"][199]["labels"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

5.4 Ensemble of   trees using LPBoost Boosting is another ensemble method used to enhance the performance of weak learners i.e. trees . 
O   B-ML     I-ML I-ML  O     B-ML    I-ML     O  O       B-ML     I-ML   O    O  O       O   O           O  O    O        O    O     O 


# Check the Tokenizer for BERT

In [52]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.is_fast

True

In [53]:
inputs = tokenizer(frst_scnd_train_thrd_test_dd['train'][199]['tokens'], is_split_into_words=True)

print(inputs.tokens(), '\n')
print(inputs.items(), '\n')
print(inputs['input_ids'], '\n')
print(inputs.word_ids(), '\n')
print(tokenizer.pad_token_id)

['[CLS]', '5', '.', '4', 'Ensemble', 'of', 'trees', 'using', 'LP', '##B', '##oos', '##t', 'Bo', '##ost', '##ing', 'is', 'another', 'ensemble', 'method', 'used', 'to', 'enhance', 'the', 'performance', 'of', 'weak', 'learn', '##ers', 'i', '.', 'e', '.', 'trees', '.', '[SEP]'] 

dict_items([('input_ids', [101, 126, 119, 125, 12618, 1104, 2863, 1606, 6400, 2064, 26459, 1204, 9326, 15540, 1158, 1110, 1330, 9525, 3442, 1215, 1106, 11778, 1103, 2099, 1104, 4780, 3858, 1468, 178, 119, 174, 119, 2863, 119, 102]), ('token_type_ids', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]) 

[101, 126, 119, 125, 12618, 1104, 2863, 1606, 6400, 2064, 26459, 1204, 9326, 15540, 1158, 1110, 1330, 9525, 3442, 1215, 1106, 11778, 1103, 2099, 1104, 4780, 3858, 1468, 178, 119, 174, 119, 2863, 119, 102] 

[None, 0, 0, 0, 1, 2, 3, 4

### The tokenizer works fine ✔

# Align sequences after the BERT-tokenization

For instance, we had the following sequence: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

The same sequence after tokenization: ['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]'

For the latter we need to add ids, labels and tokens]

In [54]:
def align_labels_with_tokens(labels, word_ids):
    """Assign -100 to None and the same label to a token if it was split"""
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id] # by default -100 is an index that is ignored in the loss function (cross entropy)
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [55]:
inputs = tokenizer(frst_scnd_train_thrd_test_dd['train'][199]['tokens'], is_split_into_words=True)
labels = frst_scnd_train_thrd_test_dd['train'][199]['labels']
print(f"ner_labels: {labels} \n")
word_ids = inputs.word_ids()
print(f"word_ids after tokenization (order in the seq): {word_ids} \n")

# initial sequence
print(f"Initial sample:\n{frst_scnd_train_thrd_test_dd['train'][199]['tokens']}\n\nIts NER-representation:\n{labels} \n")
print("________________________________________________\n")
# sequence after tokenization routine for BERT
print(f"The same sample after tokenization:\n{tokenizer(frst_scnd_train_thrd_test_dd['train'][199]['tokens'], is_split_into_words=True).tokens()}\n\nIts NER-representation:\n{align_labels_with_tokens(labels, word_ids)} \n")

ner_labels: [0, 1, 2, 2, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

word_ids after tokenization (order in the seq): [None, 0, 0, 0, 1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 19, 19, 19, 20, 21, None] 

Initial sample:
['5.4', 'Ensemble', 'of', 'trees', 'using', 'LPBoost', 'Boosting', 'is', 'another', 'ensemble', 'method', 'used', 'to', 'enhance', 'the', 'performance', 'of', 'weak', 'learners', 'i.e.', 'trees', '.']

Its NER-representation:
[0, 1, 2, 2, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

________________________________________________

The same sample after tokenization:
['[CLS]', '5', '.', '4', 'Ensemble', 'of', 'trees', 'using', 'LP', '##B', '##oos', '##t', 'Bo', '##ost', '##ing', 'is', 'another', 'ensemble', 'method', 'used', 'to', 'enhance', 'the', 'performance', 'of', 'weak', 'learn', '##ers', 'i', '.', 'e', '.', 'trees', '.', '[SEP]']

Its NER-representation:
[-100, 0, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2

## Tokenize and align the whole dataset

In [56]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'], truncation=True, is_split_into_words=True
    )
    all_labels = examples['labels']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [57]:
tokenized_datasets = frst_scnd_train_thrd_test_dd.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=frst_scnd_train_thrd_test_dd['train'].column_names,
)

Map:   0%|          | 0/1117 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

In [58]:
tokenized_datasets['train'][120]

{'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100],
 'input_ids': [101,
  1188,
  2237,
  2515,
  1103,
  24431,
  10008,
  1155,
  1103,
  11934,
  1105,
  1147,
  19218,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

**X = input_ids**

**y_true = labels**

### Our training data is ready to be fitted to BERT ✔

# Collate data

In [59]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [60]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(199, 202)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    1,    2,    2,    0,    1,    2,    2,    2,
            2,    2,    2,    0,    0,    1,    2,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
         -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1,    2,    0,    0,    0,    0,    0,
            0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    1,    2,    0,    0,    0,    0,
            0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100]])

# Evaluation

In [61]:
metric = evaluate.load('seqeval')

In [62]:
labels = frst_scnd_train_thrd_test_dd['train'][199]['labels']
labels = [label_names[i] for i in labels]
labels

['O',
 'B-ML',
 'I-ML',
 'I-ML',
 'O',
 'B-ML',
 'I-ML',
 'O',
 'O',
 'B-ML',
 'I-ML',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [63]:
predictions = labels.copy()
print(predictions)
for idx, label in enumerate(predictions):
    if idx == 3 or idx == 5:
        predictions[idx] = 'O'
print(predictions)
metric.compute(predictions=[predictions], references=[labels])

['O', 'B-ML', 'I-ML', 'I-ML', 'O', 'B-ML', 'I-ML', 'O', 'O', 'B-ML', 'I-ML', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'B-ML', 'I-ML', 'O', 'O', 'O', 'I-ML', 'O', 'O', 'B-ML', 'I-ML', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


{'ML': {'precision': 0.3333333333333333,
  'recall': 0.3333333333333333,
  'f1': 0.3333333333333333,
  'number': 3},
 'overall_precision': 0.3333333333333333,
 'overall_recall': 0.3333333333333333,
 'overall_f1': 0.3333333333333333,
 'overall_accuracy': 0.9090909090909091}

# The following function will be applied in Trainer down below to compute metrics for the whole validation set.

In [64]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [65]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

# Import BERT

In [66]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, 
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
model.config.num_labels

3

# Set train arguments

In [75]:
args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Train the model

In [77]:
start = time.time() 

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

end = time.time()

print(f'Runtime: {end - start}')

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.105906,0.581633,0.522936,0.550725,0.968709
2,No log,0.096057,0.601770,0.623853,0.612613,0.971941
3,No log,0.133957,0.563636,0.568807,0.566210,0.972381
4,0.035600,0.139458,0.534483,0.568807,0.551111,0.970912
5,0.035600,0.143096,0.556522,0.587156,0.571429,0.972234


Runtime: 2540.6725029945374


# Test the seqeval package

In [4]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'O', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
print(classification_report(y_true, y_pred))
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")

              precision    recall  f1-score   support

        MISC       1.00      1.00      1.00         1
         PER       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

Accuracy: 0.9
